# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [63]:
import re
import pandas as pd
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for key in obj.keys():
    print(f"{key} = {obj[key]}")

home_page = https://github.com/pypa/sampleproject
keywords = sample setuptools development
license = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [64]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])

p = re.compile(r'[0-9]{2}-[0-9]')

for i in obj:
    a = re.findall(p, i)
    print(a)

['19-1']
['20-4']
['20-3']


3. Разбейте текст формулировки задачи 2 на слова.

In [65]:
s = "Написать регулярное выражение,которое позволит найти номера групп студентов"
re.split(' |,', s)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

4. Найдите в тексте все последовательности 'вол', 'воз', 'вон': "вол, воз, вон.
вол воБ во8 воз вок вог во4 воХ во! воь вон" (От Калажоков З.Х.)

In [66]:
for match in re.findall(r'во[лзн]', "вол, воз, вон. вол воБ во8 воз вок вог во4 воХ во! воь вон"):
    print(match)

вол
воз
вон
вол
воз
вон


5. Напишите регулярное выражение, которое найдёт все кабинеты с трёхзначным номером: 100 - 999 в строке

'147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'.

Шаблон кабинета: ddd кабинет, где d - арабская цифра. (От Калажоков З.Х.)

In [67]:
s = '147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'
for match in re.findall(r'[0-9][0-9][0-9]', s):
    print(match)

147
843
010
514
246
572


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [68]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')
random_recipes = recipes.sample(n=5)

mx_ln_id = len(str(random_recipes['id'].max()))
mx_ln_minutes = len(str(random_recipes['minutes'].max()))


print("|" + " "*(mx_ln_id) + "id" + " "*(mx_ln_id) + "|" + " "*(mx_ln_minutes) + "minutes" + " "*(mx_ln_minutes) + "|")
print('|' + '-'*(mx_ln_id*2 + 2 + mx_ln_minutes*2 + 8) + '|')

for index, row in random_recipes.iterrows ():
    print('|' + ' '*(mx_ln_id - 2) + f'{row.id}'.ljust(mx_ln_id*2 - 2) + '|' + f'{row.minutes}'.center(7 + mx_ln_id) + '|')


|      id      |   minutes   |
|----------------------------|
|    36409     |      5      |
|    76994     |     250     |
|    162050    |      80     |
|    283045    |      45     |
|    20187     |      30     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [69]:
import pandas as pd
import xml.etree.ElementTree as ET

def show_info(recipes_data, steps_data, recipe_id):
    # Получаем информацию о рецепте по его id
    recipe = recipes_data[recipes_data['id'] == int(recipe_id)].squeeze()
    
    # Получаем шаги приготовления для данного рецепта
    recipe_steps = steps_data[steps_data['id'] == recipe_id]

    # Формируем строку с описанием рецепта
    result_str = f'"{recipe["name"]}"\n\n'
    for i, step in enumerate(recipe_steps['steps'], start=1):
        result_str += f'{i}. {step}\n'
    result_str += '----------\n'
    result_str += f'Автор: {recipe["contributor_id"]}\n'
    result_str += f'Среднее время приготовления: {recipe["minutes"]} минут'
    
    return result_str

# Загрузка данных из файла recipes_sample.csv
recipes_data = pd.read_csv('recipes_sample.csv')

# Загрузка данных из файла steps_sample.xml
prstree = ET.parse('steps_sample.xml') 
root = prstree.getroot()
all_items = [] 
for step in root.iter('recipe'):
    id = step.find('id').text 
    steps = ''
    g = 0
    for i in step.find('steps'):
        steps += f'{g}. ' + i.text + '\n'
        g += 1
    all_items.append([id, steps])

steps_data = pd.DataFrame(all_items,columns=[ 
  'id','steps'])   

# Вызов функции show_info для рецепта с id 170895
recipe_id = '170895'
recipe_info = show_info(recipes_data, steps_data, recipe_id)

# Вывод полученной строки на экран
print(recipe_info)

"leeks and parsnips  sauteed or creamed"

1. 0. clean the leeks and discard the dark green portions
1. cut the leeks lengthwise then into one-inch pieces
2. melt the butter in a medium skillet , med
3. heat
4. add the garlic and fry 'til fragrant
5. add leeks and fry until the leeks are tender , about 6-minutes
6. meanwhile , peel and chunk the parsnips into one-inch pieces
7. place in a steaming basket and steam 'til they are as tender as you prefer
8. i like them fork-tender
9. drain parsnips and add to the skillet with the leeks
10. add salt and pepper
11. gently sautee together for 5-minutes
12. at this point you can serve it , or continue on and cream it:
13. in a jar with a screw top , add the half-n-half and arrowroot
14. shake 'til blended
15. turn heat to low under the leeks and parsnips
16. pour in the arrowroot mixture , stirring gently as you pour
17. if too thick , gradually add the water
18. let simmer for a couple of minutes
19. taste to adjust seasoning , probably an ad

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [70]:
import re

pattern = r'\b(\d+) (hour|hours|minute|minutes)\b'

for step in steps_data[steps_data['id'] == '25082']['steps']:
    matches = re.findall(pattern, step)
    if matches:
        for match in matches:
            print(f'Найдено: {match[0]} {match[1]}')

Найдено: 20 minutes
Найдено: 10 minutes
Найдено: 2 hours
Найдено: 10 minutes
Найдено: 20 minutes
Найдено: 30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [71]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

# Предположим, что у нас есть объект Series с текстами описаний рецептов
descriptions = recipes['description']

# Регулярное выражение для поиска шаблона "this..., but" в начале строки
pattern = r'^this[\w\s]*,\s?but\b'

# Поиск рецептов, содержащих данный шаблон
matching_recipes = []
for i in range(descriptions.size):
    if re.search(pattern, str(descriptions[i])) != None:
        matching_recipes.append(re.search(pattern, descriptions[i]))
# Вывод количества совпадающих рецептов и 3 примеров таких рецептов
print(f"Количество рецептов с данным шаблоном: {len(matching_recipes)}")
print("\nПримеры рецептов:")
for description in matching_recipes[:3]:
    print(str(description))
    

Количество рецептов с данным шаблоном: 132

Примеры рецептов:
<re.Match object; span=(0, 44), match='this is a great meal eaten the same day ,but'>
<re.Match object; span=(0, 54), match='this was adapted from a recipe i found on the net>
<re.Match object; span=(0, 68), match='this is kind of similar to some of the other vers>


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [72]:
import re

recipe_steps = steps_data[steps_data['id'] == '72367']['steps'].to_string(index=False)

pattern = r'\s*/\s*'

modified_steps = re.sub(pattern, '/', recipe_steps)

print(modified_steps)

0. mix butter , flour , 1/3 c\n1. sugar and 1...


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [73]:
! pip install nltk

In [76]:
import nltk
from nltk.tokenize import word_tokenize
import string

unique_words = set()
steps = []

for index, row in steps_data.iterrows ():
    steps.append(row['steps'])

for step in steps:
    words = word_tokenize(step, language='english')
    for word in words:
        word = word.lower()
        if all(char.isalpha() for char in word):
            unique_words.add(word)

print(f"Количество уникальных слов среди всех рецептов: {len(unique_words)}")


Количество уникальных слов среди всех рецептов: 14922


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
descr = recipes['description']


top_5_longest = recipes.nlargest(5, 'Num Sentences')
print(top_5_longest[['Recipe', 'Num Sentences']])

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
